In [1]:
# import os
# os.chdir("../")
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# indexes = 53456
# import numpy 
# # n = 1
# # x_train_full = jnp.concatenate((jnp.array(x_train.reshape(-1,n)), jnp.array(np.array(x_train_timstamp))), axis=1)
# # x_test_full = jnp.concatenate((jnp.array(x_test.reshape(-1,n)), jnp.array(np.array(x_test_timestamp))), axis=1)
# # x_train_full.shape, x_test_full.shape

# x = torch.tensor(np.array(x_train_full)).to(torch.float32)
# y = torch.tensor(np.array(y_train)).reshape(-1).to(torch.float32)
# xt = torch.tensor(np.array(x_test_full)).to(torch.float32)
# yt = torch.tensor(np.array(y_test)).reshape(-1).to(torch.float32)

# if x.shape[0]>indexes:
#   x = x[:indexes]
#   y = y[:indexes]

In [3]:
import math
import tqdm
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

In [4]:
# import urllib.request
# import os
# from scipy.io import loadmat
# from math import floor


# # this is for running the notebook in our testing framework
# smoke_test = ('CI' in os.environ)


# # if not smoke_test and not os.path.isfile('../elevators.mat'):
# #     print('elevators\' UCI dataset...')
# #     urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', '../elevators.mat')

# X, y = torch.tensor(torch.randn(16599, 18)), torch.tensor(torch.randn(16599))

# # if smoke_test:  # this is for running the notebook in our testing framework
# #     print("hello")
# # else:
# #     data = loadmat('elevators.mat')['data']
# #     print(data.shape[0])
# #     X = torch.tensor(data[:, :-1])
# #     X = X - X.min(0)[0]
# #     X = 2 * (X / X.max(0)[0]) - 1
# #     y = data[:, -1]


# train_n = int(floor(0.8 * len(X)))
# train_x = torch.tensor(X[:train_n, :]).contiguous().to(torch.float32)
# train_y = torch.tensor(y[:train_n]).contiguous().to(torch.float32)

# test_x = torch.tensor(X[train_n:, :]).contiguous().to(torch.float32)
# test_y = torch.tensor(y[train_n:]).contiguous().to(torch.float32)

# if torch.cuda.is_available():
#     train_x, train_y, test_x, test_y = train_x, train_y, test_x, test_y

In [5]:
import urllib.request
import os
from scipy.io import loadmat
from math import floor


# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)


# if not smoke_test and not os.path.isfile('../elevators.mat'):
#     print('Downloading \'elevators\' UCI dataset...')
#     urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', '../elevators.mat')


if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(20, 3), torch.randn(20)
else:
    data = torch.Tensor(loadmat('elevators.mat')['data'])
    X = data[:, :-1]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]


train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

In [6]:
train_x.shape, train_y.shape, test_x.shape, train_x.size(-1), train_x.dtype, train_y.dtype

(torch.Size([13279, 18]),
 torch.Size([13279]),
 torch.Size([3320, 18]),
 18,
 torch.float32,
 torch.float32)

In [11]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 500))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(500, 50))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(50, 18))

feature_extractor = LargeFeatureExtractor().cuda()

In [12]:
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
import numpy as np
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.base_covar_module =  gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=18)) #+ gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=1, active_dims=(49)))*gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel(ard_num_dims=1, active_dims=(49)))
            # self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            #     gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2)),
            #     num_dims=2, grid_size=100
            # )
            self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[np.arange(0, train_x.shape[0], 10)], likelihood=likelihood)
            self.feature_extractor = feature_extractor

            # This module will scale the NN features so that they're nice values
            # self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor(x)
            # projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1
            print(projected_x.dtype)
            mean_x = self.mean_module(projected_x) #projected_
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [13]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_x, train_y, likelihood)

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [14]:
train_x.dtype, train_y.dtype

(torch.float32, torch.float32)

In [18]:
training_iterations =  60

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def train():
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x.cuda())
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y.cuda())
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()

%time train()

  0%|          | 0/60 [00:00<?, ?it/s]

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
CPU times: user 2.4 s, sys: 7.95 ms, total: 2.41 s
Wall time: 2.39 s


In [17]:
print(gpytorch.__version__)

1.7.0


In [20]:
for name, params in model.named_parameters():
  print(name, params.dtype)

likelihood.noise_covar.raw_noise torch.float32
mean_module.constant torch.float32
covar_module.base_kernel.raw_outputscale torch.float32
covar_module.base_kernel.base_kernel.raw_lengthscale torch.float32
feature_extractor.linear1.weight torch.float32
feature_extractor.linear1.bias torch.float32
feature_extractor.linear2.weight torch.float32
feature_extractor.linear2.bias torch.float32
feature_extractor.linear3.weight torch.float32
feature_extractor.linear3.bias torch.float32
feature_extractor.linear4.weight torch.float32
feature_extractor.linear4.bias torch.float32


In [19]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    preds = model(test_x)

torch.float32
torch.float32


In [20]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 0.06775031983852386
